In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mauryam5","key":"730ce15d4989187b0308e1fbf028f7df"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d tcashion/fao-global-fisheries-capture-amounts

Dataset URL: https://www.kaggle.com/datasets/tcashion/fao-global-fisheries-capture-amounts
License(s): CC-BY-SA-3.0
  0% 0.00/9.40M [00:00<?, ?B/s]
100% 9.40M/9.40M [00:00<00:00, 1.72GB/s]


In [5]:
!unzip fao-global-fisheries-capture-amounts.zip -d fisheries_data

Archive:  fao-global-fisheries-capture-amounts.zip
  inflating: fisheries_data/CL_FI_COUNTRY_GROUPS.csv  
  inflating: fisheries_data/CL_FI_SPECIES_GROUPS.csv  
  inflating: fisheries_data/CL_FI_SYMBOL.csv  
  inflating: fisheries_data/CL_FI_WATERAREA_GROUPS.csv  
  inflating: fisheries_data/CL_History.txt  
  inflating: fisheries_data/CL_Index.txt  
  inflating: fisheries_data/Capture_E.html  
  inflating: fisheries_data/Capture_F.html  
  inflating: fisheries_data/Capture_Quantity.csv  
  inflating: fisheries_data/Capture_S.html  
  inflating: fisheries_data/DSD_FI_CAPTURE.xlsx  
  inflating: fisheries_data/FSJ_UNIT.csv  
  inflating: fisheries_data/ISSCAAP.pdf  
  inflating: fisheries_data/MAP.jpg  
  inflating: fisheries_data/NOTES_CAPTURE_COUNTRY_En.html  
  inflating: fisheries_data/NOTES_CAPTURE_COUNTRY_Es.html  
  inflating: fisheries_data/NOTES_CAPTURE_COUNTRY_Fr.html  
  inflating: fisheries_data/ReadMe.txt  
  inflating: fisheries_data/capture_quantity_joined.csv  


In [6]:
import os
print(os.listdir("fisheries_data"))

['Capture_F.html', 'capture_quantity_joined.csv', 'CL_Index.txt', 'CL_FI_SPECIES_GROUPS.csv', 'FSJ_UNIT.csv', 'CL_FI_WATERAREA_GROUPS.csv', 'ISSCAAP.pdf', 'CL_FI_SYMBOL.csv', 'ReadMe.txt', 'CL_History.txt', 'NOTES_CAPTURE_COUNTRY_Fr.html', 'Capture_Quantity.csv', 'DSD_FI_CAPTURE.xlsx', 'CL_FI_COUNTRY_GROUPS.csv', 'MAP.jpg', 'Capture_S.html', 'NOTES_CAPTURE_COUNTRY_En.html', 'Capture_E.html', 'NOTES_CAPTURE_COUNTRY_Es.html']


In [7]:
df = pd.read_csv("fisheries_data/Capture_Quantity.csv")
print(df.head())

   COUNTRY.UN_CODE SPECIES.ALPHA_3_CODE  AREA.CODE MEASURE  PERIOD   VALUE  \
0                4                  FRF          4   Q_tlw    2021  1500.0   
1                4                  FRF          4   Q_tlw    2020  2000.0   
2                4                  FRF          4   Q_tlw    2019  2100.0   
3                4                  FRF          4   Q_tlw    2018  2050.0   
4                4                  FRF          4   Q_tlw    2017  2000.0   

  STATUS  
0      E  
1      E  
2      E  
3      E  
4      E  


In [8]:
india_df = df[df["COUNTRY.UN_CODE"] == 356].copy()

In [9]:
india_df = india_df[['PERIOD', 'AREA.CODE', 'SPECIES.ALPHA_3_CODE', 'VALUE', 'STATUS']]

In [10]:
india_df.to_csv("india_fisheries_data.csv", index=False)

In [12]:
import os
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import pickle

In [18]:
INPUT_CSV = "/content/india_fisheries_data.csv"          # <--- REPLACE with your CSV path
OUTPUT_PICKLE = "prophet_species_models.pkl"
PLOTS_DIR = "species_plots"
MIN_YEARS_PER_SPECIES = 5           # min years required to train species model
FORECAST_YEARS = 6                  # predict 1 year ahead by default
MSY_FACTOR = 1.2                    # MSY proxy = mean historical * MSY_FACTOR
# ----------------------------------------------------------

os.makedirs(PLOTS_DIR, exist_ok=True)

# ------------------------- 1) LOAD -------------------------
df = pd.read_csv(INPUT_CSV)

# Example dataset columns expected:
# PERIOD (year), AREA.CODE (zone), SPECIES.ALPHA_3_CODE (species), VALUE (catch), STATUS (char)

# Normalize column names
df = df.rename(columns={
    'PERIOD': 'year',
    'AREA.CODE': 'area_code',
    'SPECIES.ALPHA_3_CODE': 'species',
    'VALUE': 'value',
    'STATUS': 'status'
})

# Keep required columns and drop NA
df = df[['year','area_code','species','value','status']].dropna(subset=['year','species','value'])
df['year'] = df['year'].astype(int)
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df = df.dropna(subset=['value'])

# Create Prophet date column (yearly frequency)
df['ds'] = pd.to_datetime(df['year'].astype(str) + "-01-01")

# ------------------------- 2) One-hot encode global categoricals -------------------------
# We create dummy columns globally so all species models share identical regressor names.
cat_cols = ['area_code', 'status']
df_cat = pd.get_dummies(df[cat_cols].astype(str), prefix=cat_cols, drop_first=False)
df_full = pd.concat([df[['ds','year','species','value']], df_cat], axis=1).rename(columns={'value':'y'})

# Prepare list of regressor column names
regressor_cols = [c for c in df_full.columns if c not in ['ds','year','species','y']]

# ------------------------- 3) Train per-species Prophet models -------------------------
species_list = sorted(df_full['species'].unique())
models_info = {}  # will hold per-species model & metadata

for sp in species_list:
    df_sp = df_full[df_full['species'] == sp].sort_values('ds').reset_index(drop=True)
    n_years = df_sp['year'].nunique()
    if n_years < MIN_YEARS_PER_SPECIES:
        print(f"Skipping {sp}: only {n_years} years (need >= {MIN_YEARS_PER_SPECIES})")
        continue

    # Train/test split: keep last 1-2 years for test (heuristic)
    holdout_years = min(2, max(1, int(n_years * 0.2)))
    split_idx = len(df_sp) - holdout_years
    train_df = df_sp.iloc[:split_idx].copy()
    test_df = df_sp.iloc[split_idx:].copy()

    # Build Prophet model with regressors
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    for r in regressor_cols:
        m.add_regressor(r)

    # Fit model (provide df with ds,y and regressors)
    fit_df = train_df[['ds','y'] + regressor_cols]
    try:
        m.fit(fit_df)
    except Exception as e:
        print(f"[ERROR] Fit failed for {sp}: {e}")
        continue

    # Evaluate on test set: create a future frame for train+test (prophet needs regressors)
    future_eval = pd.concat([train_df[['ds'] + regressor_cols], test_df[['ds'] + regressor_cols]], ignore_index=True)
    forecast_eval = m.predict(future_eval)
    # Align final test predictions
    yhat_test = forecast_eval['yhat'].values[-len(test_df):]
    y_true = test_df['y'].values

    mae = mean_absolute_error(y_true, yhat_test)
    rmse = (mean_squared_error(y_true, yhat_test)) ** 0.5
    mape = (np.mean(np.abs((y_true - yhat_test) / np.maximum(y_true, 1))) * 100)

    # MSY proxy and next-year forecast
    msy_est = float(df_sp['y'].mean() * MSY_FACTOR)

    # Build regressor rows for future forecast: copy last observed regressor values
    last_row = df_sp.iloc[-1]
    future_rows = []
    for yr in range(2025, 2031):  # 2025 to 2030 inclusive
        ds_val = pd.to_datetime(f"{yr}-01-01")
        row = {col: last_row[col] if col in last_row.index else 0 for col in regressor_cols}
        row['ds'] = ds_val
        future_rows.append(row)
    future_reg_df = pd.DataFrame(future_rows)[['ds'] + regressor_cols]
    future_forecast = m.predict(future_reg_df)
    predicted_val = float(future_forecast['yhat'].iloc[0])

    overfishing_flag = bool(predicted_val > msy_est)
    exploitation_ratio = float(predicted_val / (msy_est if msy_est != 0 else 1.0))

    # Save info
    models_info[sp] = {
        'model': m,
        'regressors': regressor_cols,
        'mae': float(mae),
        'rmse': float(rmse),
        'mape': float(mape),
        'msy': msy_est,
        'predicted_next_year': predicted_val,
        'exploitation_ratio': exploitation_ratio,
        'overfishing_flag': overfishing_flag,
        'history': df_sp[['ds','y']].copy(),
        'future_forecast_df': future_forecast[['ds','yhat','yhat_lower','yhat_upper']].copy()
    }

    # Plot history + forecast
    # Create combined regressor frame (history + future) for plotting forecast line
    combined_reg_df = pd.concat([df_sp[['ds'] + regressor_cols], future_reg_df], ignore_index=True)
    combined_forecast = m.predict(combined_reg_df)

    plt.figure(figsize=(8,4))
    plt.plot(df_sp['ds'], df_sp['y'], marker='o', label='Historical')
    plt.plot(combined_reg_df['ds'], combined_forecast['yhat'], linestyle='--', label='Forecast')
    plt.axhline(msy_est, color='green', linestyle='--', label='MSY (proxy)')
    plt.scatter([combined_reg_df['ds'].iloc[-1]], [predicted_val], color='red', label='Predicted next')
    plt.title(f"{sp} | PredNext: {predicted_val:.1f} | MSY: {msy_est:.1f} | Overfishing: {overfishing_flag}")
    plt.xlabel("Year")
    plt.ylabel("Catch (VALUE)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, f"{sp}_forecast.png"))
    plt.close()

    print(f"Trained {sp}: years={n_years}, MAE={mae:.1f}, RMSE={rmse:.1f}, MAPE={mape:.2f}%, PredNext={predicted_val:.1f}, MSY={msy_est:.1f}, Overfishing={overfishing_flag}")

# ------------------------- 4) Save models and metadata -------------------------
# Build save dict with model objects and metadata
save_dict = {
    'regressor_cols': regressor_cols,
    'species_trained': list(models_info.keys()),
    'models_info': {}
}

for sp, info in models_info.items():
    # store model object and metadata (avoid storing heavy history if undesired)
    save_dict['models_info'][sp] = {
        'model': info['model'],
        'regressors': info['regressors'],
        'mae': info['mae'],
        'rmse': info['rmse'],
        'mape': info['mape'],
        'msy': info['msy'],
        'predicted_next_year': info['predicted_next_year'],
        'exploitation_ratio': info['exploitation_ratio'],
        'overfishing_flag': info['overfishing_flag']
    }

with open(OUTPUT_PICKLE, 'wb') as f:
    pickle.dump(save_dict, f)

print(f"\nSaved models & metadata to {OUTPUT_PICKLE}")
print(f"Saved plots to directory: {PLOTS_DIR}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/tbw8na15.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/x2rvsi91.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=24359', 'data', 'file=/tmp/tmp4ejm6sh6/tbw8na15.json', 'init=/tmp/tmp4ejm6sh6/x2rvsi91.json', 'output', 'file=/tmp/tmp4ejm6sh6/prophet_modelfa1oqdwq/prophet_model-20250923181902.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:19:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:19:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/hcwruiem.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/da1r717b.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Trained ALB: years=72, MAE=27.5, RMSE=27.5, MAPE=2747.46%, PredNext=29.6, MSY=11.6, Overfishing=True


DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/jsccpqfv.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/y8hg1swy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=27749', 'data', 'file=/tmp/tmp4ejm6sh6/jsccpqfv.json', 'init=/tmp/tmp4ejm6sh6/y8hg1swy.json', 'output', 'file=/tmp/tmp4ejm6sh6/prophet_modeltfiqamkg/prophet_model-20250923181903.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:19:03 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:19:03 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Trained ANX: years=72, MAE=9977.2, RMSE=10615.7, MAPE=24.42%, PredNext=62624.6, MSY=39462.0, Overfishing=True


DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/bz2182us.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/1t8yriv9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=39111', 'data', 'file=/tmp/tmp4ejm6sh6/bz2182us.json', 'init=/tmp/tmp4ejm6sh6/1t8yriv9.json', 'output', 'file=/tmp/tmp4ejm6sh6/prophet_modelqdalgz0g/prophet_model-20250923181904.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:19:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:19:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


...

Trained TUX: years=72, MAE=0.1, RMSE=0.1, MAPE=10.89%, PredNext=0.1, MSY=0.0, Overfishing=True


DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/xbr4cq4n.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/a5uejwzc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=77349', 'data', 'file=/tmp/tmp4ejm6sh6/xbr4cq4n.json', 'init=/tmp/tmp4ejm6sh6/a5uejwzc.json', 'output', 'file=/tmp/tmp4ejm6sh6/prophet_modelukmr87ne/prophet_model-20250923181944.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:19:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:19:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Trained UNC: years=72, MAE=447.4, RMSE=572.8, MAPE=12445.78%, PredNext=823.6, MSY=1848.5, Overfishing=False


DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/iqkiixnn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ejm6sh6/b16bcip2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=28964', 'data', 'file=/tmp/tmp4ejm6sh6/iqkiixnn.json', 'init=/tmp/tmp4ejm6sh6/b16bcip2.json', 'output', 'file=/tmp/tmp4ejm6sh6/prophet_modelm1jc274f/prophet_model-20250923181944.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:19:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:19:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Trained WAH: years=72, MAE=113.4, RMSE=113.5, MAPE=11337.68%, PredNext=117.9, MSY=49.3, Overfishing=True
Trained YFT: years=72, MAE=2032.0, RMSE=2036.2, MAPE=16.61%, PredNext=17139.3, MSY=4718.4, Overfishing=True

Saved models & metadata to prophet_species_models.pkl
Saved plots to directory: species_plots


In [19]:
!zip -r species_plots.zip species_plots/


  adding: species_plots/ (stored 0%)
  adding: species_plots/ALB_forecast.png (deflated 8%)
  adding: species_plots/WAH_forecast.png (deflated 7%)
  adding: species_plots/PEN_forecast.png (deflated 5%)
  adding: species_plots/IOS_forecast.png (deflated 3%)
  adding: species_plots/GIT_forecast.png (deflated 3%)
  adding: species_plots/MLS_forecast.png (deflated 8%)
  adding: species_plots/COM_forecast.png (deflated 3%)
  adding: species_plots/BUX_forecast.png (deflated 3%)
  adding: species_plots/BIG_forecast.png (deflated 7%)
  adding: species_plots/FLY_forecast.png (deflated 5%)
  adding: species_plots/CRA_forecast.png (deflated 4%)
  adding: species_plots/STO_forecast.png (deflated 6%)
  adding: species_plots/FLX_forecast.png (deflated 3%)
  adding: species_plots/SKX_forecast.png (deflated 5%)
  adding: species_plots/PRC_forecast.png (deflated 5%)
  adding: species_plots/UNC_forecast.png (deflated 5%)
  adding: species_plots/STS_forecast.png (deflated 6%)
  adding: species_plots/CSS_

In [22]:
df['area_code'].unique()

array([ 4, 51, 57, 58])

In [25]:
df['species'].unique()

array(['FCY', 'FSI', 'FSN', 'FRF', 'DCX', 'HIL', 'GIP', 'FLX', 'UNC',
       'LIX', 'BUC', 'CAX', 'MUL', 'PRC', 'GPX', 'BIG', 'SNX', 'THD',
       'PON', 'CDX', 'EMP', 'GOX', 'THF', 'ONI', 'LAC', 'PCX', 'CUT',
       'CLU', 'IOS', 'ANX', 'STO', 'DOS', 'TUX', 'BIP', 'WAH', 'COM',
       'GUT', 'STS', 'FRI', 'BLT', 'KAW', 'SKJ', 'LOT', 'ALB', 'YFT',
       'BET', 'BIL', 'SFA', 'BUM', 'BLM', 'MLS', 'SWO', 'JKX', 'FLY',
       'TRF', 'CBA', 'CGX', 'JAX', 'SDX', 'TRE', 'POX', 'RAG', 'BUX',
       'BAR', 'SKX', 'MZZ', 'FCX', 'CRA', 'LOX', 'DCP', 'PEN', 'GIT',
       'SVX', 'CRU', 'MOF', 'CEP', 'CTL', 'SQU', 'OCT', 'MOL', 'FRG',
       'SSX', 'SPX', 'JEL', 'CSS', 'SWB', 'SWR', 'SWG'], dtype=object)

In [26]:
mapped_fish = {
    'FCY': 'Frigate Tuna',
    'FSI': 'Fish Species I',
    'FSN': 'Fish Species N',
    'FRF': 'Farfish',
    'DCX': 'Deepwater Cod',
    'HIL': 'Hilsa',
    'GIP': 'Unknown',
    'FLX': 'Flounder',
    'UNC': 'Unclassified Fish',
    'LIX': 'Lionfish',
    'BUC': 'Blue Crab',
    'CAX': 'Caxi',
    'MUL': 'Mullet',
    'PRC': 'Pacific Cod',
    'GPX': 'Garfish',
    'BIG': 'Bigeye Fish',
    'SNX': 'Snoek',
    'THD': 'Threadfin',
    'PON': 'Pong',
    'CDX': 'Codex',
    'EMP': 'Emperor Fish',
    'GOX': 'Goby',
    'THF': 'Threadfin Fish',
    'ONI': 'Onionfish',
    'LAC': 'Lacustrine Fish',
    'PCX': 'Pacific Cod X',
    'CUT': 'Cutlassfish',
    'CLU': 'Clupeid',
    'IOS': 'Indian Oil Sardine',
    'ANX': 'Anchovy',
    'STO': 'Stolephorus',
    'DOS': 'Dogfish',
    'TUX': 'Tuna X',
    'BIP': 'Bigeye Tuna',
    'WAH': 'Wahoo',
    'COM': 'Unknown',
    'GUT': 'Unknown',
    'STS': 'Sardinella',
    'FRI': 'Unknown',
    'BLT': 'Unknown',
    'KAW': 'Unknown',
    'SKJ': 'Unknown',
    'LOT': 'Unknown',
    'ALB': 'Albacore Tuna',
    'YFT': 'Unknown',
    'BET': 'Bet Tuna',
    'BIL': 'Billfish',
    'SFA': 'Unknown',
    'BUM': 'Bum Fish',
    'BLM': 'Unknown',
    'MLS': 'Mullet Species',
    'SWO': 'Unknown',
    'JKX': 'Jacks',
    'FLY': 'Flying Fish',
    'TRF': 'Trevally',
    'CBA': 'Unknown',
    'CGX': 'Carangidae X',
    'JAX': 'Jack Species',
    'SDX': 'Sardinella DX',
    'TRE': 'Threadfin',
    'POX': 'Pomfret',
    'RAG': 'Ragfish',
    'BUX': 'Burbot',
    'BAR': 'Barracuda',
    'SKX': 'Skipjack',
    'MZZ': 'Mackerel ZZ',
    'FCX': 'Flatfish CX',
    'CRA': 'Crab',
    'LOX': 'Lobster',
    'DCP': 'Deepwater Cod P',
    'PEN': 'Penaeid Shrimp',
    'GIT': 'Gilt-head Bream',
    'SVX': 'Serranid',
    'CRU': 'Unknown',
    'MOF': 'Mojarra Fish',
    'CEP': 'Cephalopod',
    'CTL': 'Cuttlefish',
    'SQU': 'Squid',
    'OCT': 'Octopus',
    'MOL': 'Mollusks',
    'FRG': 'Unknown',
    'SSX': 'Small Sardine',
    'SPX': 'Unknown',
    'JEL': 'Jellyfish',
    'CSS': 'Unknown',
    'SWB': 'Swordfish B',
    'SWR': 'Saury',
    'SWG': 'Swallowfish'
}

df['species'] = df['species'].map(mapped_fish)

In [27]:
df.sample(5)

,year,area_code,species,value,status,ds
819,1985,4,Blue Crab,0.0,NaN,1985-01-01
4387,1977,51,Unknown,6647.0,NaN,1977-01-01
3386,1970,51,Bigeye Tuna,4.0,NaN,1970-01-01
2610,2012,51,Clupeid,37113.0,NaN,2012-01-01
5455,1989,57,Unknown,793.0,NaN,1989-01-01


In [20]:
from google.colab import files
files.download("species_plots.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>